<a href="https://colab.research.google.com/github/hyeokkukim/python-for-finance/blob/main/Value_at_Rixk(VaR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parametric VaR


In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 298 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


### Single-asset Portfolio

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf


ms = yf.download('MSFT',start='2021-01-01',end = '2022-08-16')[['Close']]
ms['log_close'] = np.log(ms['Close'])
ms['change'] = ms['log_close'].pct_change()
ms = ms.dropna()
ms.head()

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Close,log_close,change
Date,,,
2021-01-05,217.899994,5.384036,0.000179
2021-01-06,212.250000,5.357765,-0.004879
2021-01-07,218.289993,5.385824,0.005237
2021-01-08,219.619995,5.391899,0.001128
2021-01-11,217.490005,5.382153,-0.001808


In [ ]:
std = ms['change'].std() #표준편차 계산
std_daily = std/np.sqrt(len(ms.index)) #1일 표준편차 계산

from scipy.stats import norm
CL = 0.99
z_score = norm.ppf(CL)

print('1day VaR of Microsoft: ',z_score * std_daily)

1day VaR of Microsoft:  0.00035051397638420206


In [ ]:
N = 20
initial_P = 100000000
VaR = z_score * std_daily * initial_P * np.sqrt(N)

print('99%신뢰수준에서 마이크로소프트에 1억 투자시 20일동안 최대손실가능액: ',round(VaR),'원')

99%신뢰수준에서 마이크로소프트에 1억 투자시 20일동안 최대손실가능액:  156755 원


### Multi-asset Portfolio

In [ ]:
tickers = ['AAPL','MSFT','BRK-B','DIS']
w = np.array([0.24,0.3,0.15,0.3]) #포트폴리오 가중치
df = yf.download(tickers,start='2021-01-01',end = '2022-08-16')[['Close']]
df = np.log(df)
df = df.pct_change()
df = df.dropna()
df.head()

[*********************100%***********************]  4 of 4 completed


Close                              
                AAPL     BRK-B       DIS      MSFT
Date                                              
2021-01-05  0.002527 -0.000792  0.000824  0.000179
2021-01-06 -0.007023  0.002254  0.000734 -0.004879
2021-01-07  0.006931  0.002072 -0.000582  0.005237
2021-01-08  0.001763  0.000904  0.000119  0.001128
2021-01-11 -0.004817 -0.000471  0.000431 -0.001808

In [ ]:
cov = df.cov() #포트폴리오 공분산 계산
std = np.sqrt(w.T.dot(cov).dot(w))
std_daily = std/np.sqrt(len(df.index)) #1일 표준편차 계산
z_score = norm.ppf(0.99)

print('Covariance of Portfolio: ',cov)
print('------------------------------')
print('1day VaR of Portfolio: ',z_score * std_daily)

Covariance of Portfolio:                  Close                              
                 AAPL     BRK-B       DIS      MSFT
Close AAPL   0.000013  0.000003  0.000006  0.000008
      BRK-B  0.000003  0.000004  0.000004  0.000002
      DIS    0.000006  0.000004  0.000014  0.000005
      MSFT   0.000008  0.000002  0.000005  0.000009
------------------------------
1day VaR of Portfolio:  0.0002718667515968532


In [ ]:
N = 20
initial_P = 100000000
VaR = z_score * std_daily * initial_P * np.sqrt(N)

print('99%신뢰수준에서 포트폴리오에 1억 투자시 20일동안 VaR: ',round(z_score * std_daily * np.sqrt(N) * 100,3),'%')
print('99%신뢰수준에서 포트폴리오에 1억 투자시 20일동안 최대손실가능액: ',round(VaR),'원')

99%신뢰수준에서 포트폴리오에 1억 투자시 20일동안 VaR:  0.122 %
99%신뢰수준에서 포트폴리오에 1억 투자시 20일동안 최대손실가능액:  121583 원


## Non-parametric VaR 

### 포트폴리오 생성

In [ ]:
tickers = ['AAPL','MSFT','BRK-B','DIS']
w = np.array([0.24,0.3,0.15,0.3]) #포트폴리오 가중치
df = yf.download(tickers,start='2021-01-01',end = '2022-08-16')[['Close']]
df = np.log(df)
df = df.pct_change()
df = df.dropna()
df.head()

[*********************100%***********************]  4 of 4 completed


Close                              
                AAPL     BRK-B       DIS      MSFT
Date                                              
2021-01-05  0.002527 -0.000792  0.000824  0.000179
2021-01-06 -0.007023  0.002254  0.000734 -0.004879
2021-01-07  0.006931  0.002072 -0.000582  0.005237
2021-01-08  0.001763  0.000904  0.000119  0.001128
2021-01-11 -0.004817 -0.000471  0.000431 -0.001808

In [ ]:
N = 20
trading_days = len(df.index)
mean_p = df.mean().to_numpy().dot(w) #portfolio expected return
cov = df.cov() #포트폴리오 공분산 계산
std = np.sqrt(w.T.dot(cov).dot(w)) #포트폴리오 표준편차
iterations = 50000 #반복횟수

### Monte Carlo Simulation

In [ ]:
def MC_VaR(expected_return, standard_deviation, T, iterations):
    
    values = 1- np.exp((expected_return - 0.5 * standard_deviation ** 2) * T + standard_deviation * np.sqrt(T) * np.random.standard_normal(iterations))
    
    return values


var = MC_VaR(expected_return = mean_p, standard_deviation=std, T = N/trading_days, iterations = iterations)
var

array([ 1.22434887e-04, -3.62751226e-04, -1.25419236e-03, ...,
       -6.90252748e-04,  2.89555351e-06, -5.95241541e-04])

In [ ]:
var = pd.DataFrame(var)
var.columns = ['var']
var = var.sort_values('var')
var = var.reset_index()
var

,index,var
0,4447,-0.002270
1,17921,-0.002034
2,24475,-0.001988
3,8599,-0.001971
4,12805,-0.001962
...,...,...
49995,47740,0.001960
49996,1408,0.002037
49997,42949,0.002063
49998,4608,0.002174


In [ ]:
q_99 = var['var'][iterations*0.01]
q_95 = var['var'][iterations*0.05]
q_90 = var['var'][iterations*0.1]

In [ ]:
print(q_99,q_95,q_90)
print(100000000*q_99,100000000*q_95,100000000*q_90)

-0.0012308193939563061 -0.0008705227170671126 -0.0006756099177160912
-123081.93939563062 -87052.27170671125 -67560.99177160913


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=var['var']))

#fig.add_trace(go.Scatter(x = [q_99, q_99],y = [0,800],mode = 'lines',line_color = 'white',line_width = 0.4))
fig.add_vline(x=q_99, line_color = '#ba1141',line_width = 0.7)
fig.add_vline(x=q_95, line_color = '#ff5c8a',line_width = 0.7)
fig.add_vline(x=q_90, line_color = '#Ff99b9',line_width = 0.7)

fig.update_layout(
        title = dict(text = 'Non-parametric VaR Quantile Distribution',font = dict(size=15,color='white')),
   #     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.93),
        autosize = True,
        showlegend = False,
        font = dict(size = 8, color = 'white'),
        plot_bgcolor = '#0d0c1d',paper_bgcolor='#0d0c1d',
        width=700,height=450,
        xaxis = dict(title = 'VaR', showline = True, showgrid = False, showticklabels = True, 
                    linecolor = 'white', linewidth = 0.8,ticks = 'outside'),
        yaxis = dict(title = 'Count', showline = True, showgrid = False, showticklabels = True, 
                    linecolor = 'white', linewidth = 0.8,ticks = 'outside'),
        annotations=[
        dict(xref='paper',yref='paper',x=1.11,y=-0.18,align='right',
            text='Source: yahoofinance',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1.11,y=-0.218,align='right',
            text='Data period: 2021.01.01 - 2022.08.16',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1.11,y=1.27,align='right',
            text='Information',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1.11,y=1.24,align='right',
            text='Linkedin: ~/chacehkk',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1.11,y=1.21,align='right',
            text='Medium: ~/@hyksun2015',showarrow=False,font=dict(size=8,color='grey'))],

        )
fig.show()

In [ ]:
var = pd.DataFrame(var)[0]

In [ ]:
std = np.sqrt(w.T.dot(cov).dot(w))

In [ ]:
vol = std * np.sqrt(len(df.index))

In [ ]:
x0

array([-0.01235826,  0.95320459, -1.06066163, ..., -0.56869372,
       -0.47462178,  1.21267375])